In [2]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
env=AngryBird()

# 6, 30, 60, 100 coeffs

In [7]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

sqr2 = 1 / np.sqrt(2)
rot = np.array([[sqr2, -sqr2], [sqr2, sqr2]])
rot2 = np.array([[sqr2, sqr2], [-sqr2, sqr2]])

vents = np.load("vents.npy")
nul = np.zeros(2)

pos = np.load("mean_states_1440.npy")

def main(state, coeffs):
    t = int(state[0])
    x1, x2 = np.matmul(rot, state[1:] - pos[t])
    u1 = np.array([x1*(x1>0), x2*(x2>0)])
    u2 = np.array([x1*(x1<0), x2*(x2<0)])
    return np.matmul(rot2, coeffs[t,:2]*u1 + coeffs[t,2:4]*u2 + coeffs[t,4:6] + coeffs[t,6:8]*u1**2 + coeffs[t,8:10]*u2**2)

def single_process(i, x):
    env = AngryBird()
    state = env.reset(False)
    env.vent = vents[i]
    for j in range(10):
        state, _, _, _ = env.step(main(state, x))
    env.step(nul)
    return np.sum(env.cost)

In [8]:
coeffs_temp = np.load("c_60.npy")
coeffs = np.zeros((10, 10))
for i in range(10):
    coeffs[i][:6] = coeffs_temp[i]

In [11]:
%%time
from scipy.optimize import minimize
counter = 0
def to_minimize(x):
    global counter
    counter += 1
    print(counter, end="\r")
    x = x.reshape(10, 10)
    return sum(Parallel(n_jobs=num_cores)(delayed(single_process)(i, x) for i in range(1000)))
minimize(to_minimize, coeffs.reshape(-1), tol=1e-3)

CPU times: user 3h 29min 51s, sys: 33min 57s, total: 4h 3min 49s
Wall time: 9h 38min 50s


      fun: 112721.19497438504
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.35726379e-03, -3.52062271e-05,  5.07292849e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.52062271e-05,  1.65051650e-05,  1.93261285e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.07292849e-05,  1.93261285e-05,  9.96634940e-04]])
      jac: array([ 0.        ,  0.        ,  0.        ,  0.        , -0.20703125,
        0.02832031,  0.        ,  0.        ,  0.        ,  0.        ,
        0.19824219,  0.015625  ,  0.01660156,  

In [12]:
coeffs = np.array([-1.64539390e+00, -1.04825820e-01, -1.62947623e+00, -2.28164270e-01,
        3.84130362e-02,  5.13647295e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.43747209e+00, -1.15658582e-01,
       -1.62912943e+00, -1.19658273e-01,  2.02934574e-01,  5.33660888e-01,
       -1.16427724e-01, -3.54146297e-02, -3.98244241e-02,  1.41959735e-01,
       -1.40722987e+00, -1.39311688e-01, -1.41893928e+00, -5.89710841e-02,
        1.80286537e-01,  5.46192406e-01, -2.19436699e-02, -1.92338203e-03,
        3.63472016e-02,  7.62719783e-02, -1.55857126e+00, -1.39471855e-01,
       -1.40074612e+00, -1.46693409e-01,  4.34267391e-01,  5.35043931e-01,
        3.04523321e-02,  2.96350337e-03,  5.16547879e-03, -1.84120189e-02,
       -1.51193388e+00, -1.60185777e-01, -1.55230259e+00, -1.74231971e-01,
        3.07098235e-01,  5.48237959e-01,  1.10096532e-02,  8.27679339e-03,
       -2.48824335e-02, -2.68162149e-02, -1.56473666e+00, -1.92192060e-01,
       -1.44728699e+00, -1.85162066e-01,  4.50005045e-01,  5.63081961e-01,
        9.89758334e-03,  8.96539390e-03,  2.46447442e-02, -1.92494265e-02,
       -1.53125017e+00, -2.62876713e-01, -1.56432642e+00, -2.24110993e-01,
        4.46805048e-01,  5.70608304e-01, -2.56296593e-03,  3.08799351e-02,
        4.96503261e-03, -2.18057452e-02, -1.58967345e+00, -3.14260397e-01,
       -1.68785931e+00, -2.87748893e-01,  3.86337047e-01,  5.65170606e-01,
        1.30446728e-04,  3.25710501e-02, -4.93166462e-03, -2.66810531e-02,
       -1.66786168e+00, -3.87555763e-01, -1.78983613e+00, -3.40199932e-01,
        4.54848376e-01,  5.66748093e-01, -6.04062606e-03,  2.46704151e-02,
       -2.50123377e-02,  1.14279451e-03, -2.01170785e+00, -5.16337533e-01,
       -1.38302781e+00, -5.61684781e-01,  9.52384935e-01,  5.66421609e-01,
       -2.69766319e-02, -1.73729783e-02, -4.96114225e-03, -2.40683470e-02]).reshape(10, 10)

In [13]:
def to_minimize(x):
    return sum(Parallel(n_jobs=num_cores)(delayed(single_process)(i, x) for i in range(10000)))
to_minimize(coeffs)

1122509.019952249

In [14]:
np.save("c_100", coeffs)